<a href="https://colab.research.google.com/github/dingzhang2023/problem-solving-practice/blob/colab/Dijkstra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Dijkstra

Dijkstra's Algorithm Introduction

> Define $ g[i][j] $ as the weight of the edge from node $i$ to node $ j $. If there is no edge from $i$ to $j$, then $ g[i][j] = \infty $.

> Define $ \text{dis}[i] $ as the shortest path length from node $k$ to node $ i $. Initially, $ \text{dis}[k] = 0 $ and all other $ \text{dis}[i] = \infty $ indicating it has not been calculated yet.

Our goal is to compute the final $ \text{dis} $ array.

1. Firstly, update the shortest path from node $ k $ to its neighbors $ y $, i.e., update $ \text{dis}[y] $ to $ g[k][y] $.
2. Then, take the minimum value of $ \text{dis}[i] $ except for node $ k $, suppose the minimum value corresponds to node $ 3 $. At this point, it can be asserted that $ \text{dis}[3] $ is the shortest path length from $ k $ to $ 3 $, and there cannot be any other shorter paths from $ k $ to $ 3 $. Proof by contradiction: Assuming there exists a shorter path, then we must go from $ k $ through a node $ u $, where $ \text{dis}[u] $ is smaller than $ \text{dis}[3] $, and then through some edges to reach $ 3 $, obtaining a smaller $ \text{dis}[3] $. But $ \text{dis}[3] $ is already the minimum, and there are no negative edge weights in the graph, so $ u $ does not exist, which is contradictory. Therefore, the original proposition holds true, and we obtain the final value of $ \text{dis}[3] $.
3. Update $ \text{dis}[y] $ using the edge weight $ g[3][y] $: if $ \text{dis}[3] + g[3][y] < \text{dis}[y] $, then update $ \text{dis}[y] $ to $ \text{dis}[3] + g[3][y] $, otherwise do not update.
4. Repeat the above process for all nodes except $ k $ and $ 3 $.
5. By mathematical induction, this approach can obtain the shortest path for each node. When the shortest paths for all nodes are determined, the algorithm ends.

**Approach 1: Naive Dijkstra (Suitable for Dense Graphs)**

For this problem, when calculating the shortest paths, if we find that the current minimum shortest path is equal to $ \infty $, it means that there are nodes that cannot be reached, and we can terminate the algorithm early, returning $ \text -1 $.

If all nodes can be reached, return $ \text max(dis) $.

For this question, node numbering should start from 0, substracting 1 from node number.

#### Template Problem
[743. Network Delay Time](https://leetcode.com/problems/network-delay-time/description/)



##### Method I: Naive Dijkstra

- Time complexity: $ O(n^2) $.
- Space complexity: $ O(n^2) $.

In [ ]:
class Solution:
    def networkDelayTime(self, times: List[List[int]], n: int, k: int) -> int:
        g = [[inf for _ in range(n)] for _ in range(n)] # Adjacency Matrix
        for x, y, d in times:
            g[x - 1][y - 1] = d # node numbering starts from 0

        dis = [inf] * n
        ans = dis[k - 1] = 0
        visited = [False] * n

        while True:
            x = -1
            for i, ok in enumerate(visited):
                if not ok and (x < 0 or dis[i] < dis[x]):
                    x = i # find the shortest edge
            if x < 0:
                return ans
            if dis[x] == inf: # cant reach a node
                return -1
            ans = dis[x]
            visited[x] = True
            for y, d in enumerate(g[x]):
                dis[y] = min(dis[y], dis[x] + d)

- Method II: Heap-Optimized Dijkstra (Suitable for Sparse Graphs)

The process of finding the minimum value can be quickly accomplished using a min-heap:

1. Initially, enqueue the tuple $ (\text{dis}[k], k) $ into the heap.
2. When a node $ x $ is dequeued for the first time, $ \text{dis}[x] $ is the minimum shortest path found so far.
3. When updating $ \text{dis}[y] $, enqueue the tuple $ (\text{dis}[y], y) $ into the heap.
4. Note that if a node $ x $ is dequeued multiple times before it's actually processed, there may be multiple occurrences of $ x $ in the heap, and the $ \text{dis}[x] $ values in the tuples containing $ x $ are all distinct (because we only enqueue a tuple when we find a shorter path).

Therefore, the use of the done array in Approach 1 can be eliminated. Instead, we compare the dequeued shortest path value (denoted as $ d_x $) with the current $ \text{dis}[x] $. If $ d_x > \text{dis}[x] $, it means that node $ x $ has been dequeued before, and we have already updated the shortest paths of $ x $'s neighbors, so we can skip this iteration and continue the outer loop.

In [ ]:
class Solution:
    def networkDelayTime(self, times: List[List[int]], n: int, k: int) -> int:
        g = [[] for _ in range(n)]  # Adjacecy Matrix
        for x, y, d in times:
            g[x - 1].append((y - 1, d))

        dis = [inf] * n
        dis[k - 1] = 0
        h = [(0, k - 1)]
        while h:
            dx, x = heappop(h)
            if dx > dis[x]:  # x is out of heap
                continue
            for y, d in g[x]:
                new_dis = dx + d
                if new_dis < dis[y]:
                    dis[y] = new_dis  # got the shortest path neighbor
                    heappush(h, (new_dis, y))
        mx = max(dis)
        return mx if mx < inf else -1